## Mounting Google Drive

In [1]:
from google.colab import drive
import os
import json

drive.mount('/content/drive')
audio_folder = "/content/drive/MyDrive/multimodal_emotion_recognition/data"

Mounted at /content/drive


## Create MFCC Dataset

In [44]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset

class MFCCDataset(Dataset):
    def __init__(self, mfcc_dir):
        self.mfcc_dir = mfcc_dir
        self.files = [f for f in os.listdir(mfcc_dir) if f.endswith('.npy')]
        self.label_map = {
            'neutral': 0,
            'calm': 1,
            'happy': 2,
            'sad': 3,
            'angry': 4,
            'fearful': 5,
            'disgust': 6,
            'surprised': 7
        }

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        mfcc = np.load(os.path.join(self.mfcc_dir, file))
        label_str = file.split("_")[0]
        label = self.label_map[label_str]
        mfcc = torch.tensor(mfcc, dtype=torch.float32).unsqueeze(0)
        return mfcc, label


## Define the CNN

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MFCCCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(MFCCCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(32 * 10 * 50, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [52]:
from torch.utils.data import DataLoader
import torch

# Paths and parameters
mfcc_dir = "/content/drive/MyDrive/multimodal_emotion_recognition/mfccs"
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and loader
dataset = MFCCDataset(mfcc_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


## Train-Validate Split

In [53]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation datasets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [67]:
# Model
model = MFCCCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

In [68]:
num_epochs = 20

for epoch in range(num_epochs):  # Number of epochs
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Now validate after every epoch
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # No gradients needed during evaluation
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_epoch_loss = val_loss / len(val_dataloader)
    val_epoch_accuracy = 100 * val_correct / val_total

    print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_accuracy:.2f}%")


Epoch [1/20], Loss: 3.2352, Accuracy: 23.96%
Validation Loss: 1.7691, Validation Accuracy: 35.76%
Epoch [2/20], Loss: 1.6059, Accuracy: 40.02%
Validation Loss: 1.6184, Validation Accuracy: 39.24%
Epoch [3/20], Loss: 1.3734, Accuracy: 48.96%
Validation Loss: 1.4892, Validation Accuracy: 43.40%
Epoch [4/20], Loss: 1.1259, Accuracy: 60.50%
Validation Loss: 1.4323, Validation Accuracy: 48.61%
Epoch [5/20], Loss: 0.9391, Accuracy: 67.01%
Validation Loss: 1.2894, Validation Accuracy: 53.47%
Epoch [6/20], Loss: 0.7264, Accuracy: 76.22%
Validation Loss: 1.2770, Validation Accuracy: 53.12%
Epoch [7/20], Loss: 0.6703, Accuracy: 77.86%
Validation Loss: 1.2436, Validation Accuracy: 59.03%
Epoch [8/20], Loss: 0.5034, Accuracy: 84.90%
Validation Loss: 1.1748, Validation Accuracy: 58.68%
Epoch [9/20], Loss: 0.4658, Accuracy: 86.11%
Validation Loss: 1.1535, Validation Accuracy: 59.38%
Epoch [10/20], Loss: 0.3877, Accuracy: 88.45%
Validation Loss: 1.2845, Validation Accuracy: 57.64%
Epoch [11/20], Loss